# What 5,500 Hotels Teach Us About Value for Money
### A Data-Driven Analysis of Booking.com & TripAdvisor

**Author:** Giorgio Vernarecci - Data Analyst  
**Dataset:** [Hotel Dataset: Rates, Reviews & Amenities (6k+)](https://www.kaggle.com/datasets/joyshil0599/hotel-dataset-rates-reviews-and-amenities5k) - Kaggle, CC0 License  
**Tools:** Python, pandas, numpy, scipy, matplotlib

---

## Project Overview

Having worked in hospitality for 6 years, I wanted to look at the hotel industry from a data perspective. This analysis explores **5,500+ hotels** from Booking.com and TripAdvisor to answer a core question:

> **Does paying more for a hotel actually get you a better experience?**

Along the way, I discovered insights about room quality, destination value, overpriced hotels, and guest review behavior.

## 1. Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import warnings
warnings.filterwarnings('ignore')

# Dark theme for all charts
plt.rcParams.update({
    'figure.facecolor': '#0A1628', 'axes.facecolor': '#0A1628',
    'axes.edgecolor': '#1a2d4a', 'axes.labelcolor': '#8899AA',
    'xtick.color': '#8899AA', 'ytick.color': '#8899AA',
    'text.color': '#FFFFFF', 'font.size': 12,
    'axes.titlesize': 16, 'axes.titleweight': 'bold',
})

ACCENT, RED, YELLOW, SUBTLE, CARD = '#00D4AA', '#FF6B6B', '#F1C40F', '#8899AA', '#132039'

print('Libraries loaded successfully.')

In [ ]:
booking_raw = pd.read_csv('data/raw/booking_hotel.csv', encoding='latin1')
tripadvisor_raw = pd.read_csv('data/raw/tripadvisor_room.csv', encoding='latin1', on_bad_lines='skip')

print(f'Booking.com raw: {booking_raw.shape[0]} rows, {booking_raw.shape[1]} columns')
print(f'TripAdvisor raw: {tripadvisor_raw.shape[0]} rows, {tripadvisor_raw.shape[1]} columns')
print(f'\nBooking columns: {list(booking_raw.columns)}')
print(f'TripAdvisor columns: {list(tripadvisor_raw.columns)}')

## 2. Data Cleaning

Both datasets required significant cleaning:
- Column names had extra spaces and inconsistent formatting
- Prices contained corrupted currency symbols (encoding issue) and comma separators
- Prices were in **BDT (Bangladeshi Taka)** — converted to EUR at approximately 120 BDT = 1 EUR
- TripAdvisor hotel names had numbering prefixes (e.g. "1. Hotel Name")
- Review counts had comma separators

**Choices made:**
- Removed hotels with missing prices or ratings
- Filtered price outliers: kept EUR 5 - EUR 10,000 per night
- The BDT/EUR rate is approximate — the exact currency in the dataset is ambiguous, but the relative comparisons remain valid

In [ ]:
# === BOOKING CLEANING ===
booking = booking_raw.copy()
booking.columns = ['hotel_name', 'location', 'rating', 'review_score',
                    'num_reviews', 'room_score', 'room_type', 'bed_type', 'price_bdt']

booking['price_bdt'] = (booking['price_bdt'].astype(str)
                        .str.replace(r'[^\d]', '', regex=True)
                        .replace('', pd.NA).astype(float))
booking['price_eur'] = (booking['price_bdt'] / 120).round(2)
booking['rating'] = pd.to_numeric(booking['rating'], errors='coerce')
booking['num_reviews'] = pd.to_numeric(booking['num_reviews'].astype(str)
                                       .str.replace(',', '').str.strip(), errors='coerce').astype('Int64')

for col in ['hotel_name', 'location', 'review_score', 'room_type', 'bed_type']:
    booking[col] = booking[col].str.strip()

booking['source'] = 'Booking.com'
booking = booking.dropna(subset=['price_bdt', 'rating'])
booking = booking[(booking['price_eur'] >= 5) & (booking['price_eur'] <= 10000)]

# === TRIPADVISOR CLEANING ===
tripadvisor = tripadvisor_raw.copy()
tripadvisor.columns = ['hotel_name', 'price_bdt', 'num_reviews', 'comment']
tripadvisor['hotel_name'] = tripadvisor['hotel_name'].str.replace(r'^\d+\.\s*', '', regex=True).str.strip()
tripadvisor['price_bdt'] = pd.to_numeric(tripadvisor['price_bdt'].astype(str)
                                          .str.replace(r'[^\d]', '', regex=True)
                                          .replace('', pd.NA), errors='coerce')
tripadvisor['price_eur'] = (tripadvisor['price_bdt'] / 120).round(2)
tripadvisor['num_reviews'] = pd.to_numeric(tripadvisor['num_reviews'].astype(str)
                                            .str.replace(',', '').str.strip(), errors='coerce').astype('Int64')
tripadvisor['comment'] = tripadvisor['comment'].str.strip()
tripadvisor['source'] = 'TripAdvisor'
tripadvisor = tripadvisor.dropna(subset=['price_bdt'])
tripadvisor = tripadvisor[(tripadvisor['price_eur'] >= 5) & (tripadvisor['price_eur'] <= 10000)]

print(f'Booking cleaned: {len(booking)} rows')
print(f'TripAdvisor cleaned: {len(tripadvisor)} rows')

## 3. Data Validation

Before proceeding with analysis, let's validate the cleaned data.

In [ ]:
print('=== BOOKING VALIDATION ===')
print(f'Rows removed during cleaning: {len(booking_raw) - len(booking)} ({(len(booking_raw) - len(booking)) / len(booking_raw) * 100:.1f}%)')
print(f'Missing values per column:')
print(booking.isnull().sum().to_string())
print(f'\nDuplicate hotel names: {booking.duplicated(subset=["hotel_name", "location"]).sum()}')
print(f'Rating range: {booking["rating"].min()} - {booking["rating"].max()}')
print(f'Price EUR range: {booking["price_eur"].min()} - {booking["price_eur"].max()}')
print(f'Unique locations: {booking["location"].nunique()}')

print(f'\n=== TRIPADVISOR VALIDATION ===')
print(f'Rows removed (cleaning + bad lines): {5330 - len(tripadvisor)}')
print(f'Empty comments: {(tripadvisor["comment"].isna() | (tripadvisor["comment"] == "")).sum()}')
print(f'Price EUR range: {tripadvisor["price_eur"].min()} - {tripadvisor["price_eur"].max()}')

print(f'\n=== PRICE COMPARISON ===')
print(f'Booking median price: EUR {booking["price_eur"].median():,.0f}')
print(f'TripAdvisor median price: EUR {tripadvisor["price_eur"].median():,.0f}')
print(f'Note: The large price difference suggests the datasets cover different market segments.')
print(f'Booking skews toward luxury/premium; TripAdvisor skews toward budget/mid-range.')

---
## 4. Finding 1 — Paying More Does NOT Guarantee a Better Experience

The core question: is there a meaningful correlation between price and guest satisfaction?

In [ ]:
# Statistical test
corr_p, p_pearson = stats.pearsonr(booking['price_eur'], booking['rating'])
corr_s, p_spearman = stats.spearmanr(booking['price_eur'], booking['rating'])
print(f'Pearson correlation:  r = {corr_p:.4f}  (p = {p_pearson:.2e})')
print(f'Spearman correlation: r = {corr_s:.4f}  (p = {p_spearman:.2e})')
print(f'\nStatistically significant but practically weak.')
print(f'Price explains less than 4% of the variance in ratings.')

In [ ]:
booking['price_bracket'] = pd.cut(booking['price_eur'],
    bins=[0, 500, 1000, 2000, 5000, 10000],
    labels=['<500', '500-1k', '1k-2k', '2k-5k', '5k+'])

pb = (booking.groupby('price_bracket', observed=True)
      .agg(mean_rating=('rating', 'mean'), count=('hotel_name', 'count'))
      .reset_index())

fig, ax = plt.subplots(figsize=(12, 7))
colors = [RED if r < 8.1 else YELLOW if r < 8.25 else ACCENT for r in pb['mean_rating']]
bars = ax.bar(pb['price_bracket'].astype(str), pb['mean_rating'], color=colors, alpha=0.85, width=0.55)
for bar, rating, count in zip(bars, pb['mean_rating'], pb['count']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03,
            f'{rating:.2f}', ha='center', fontsize=15, fontweight='bold', color='#FFFFFF')
    ax.text(bar.get_x() + bar.get_width()/2, 7.55,
            f'n={count}', ha='center', fontsize=10, color=SUBTLE)
ax.set_ylim(7.5, 8.7)
ax.set_title('Paying More Does NOT Guarantee a Better Experience', fontsize=18, pad=20)
ax.set_xlabel('Price per Night (EUR)', fontsize=13)
ax.set_ylabel('Average Rating', fontsize=13)
ax.text(0.5, 0.92, 'Only +0.28 rating difference across a 10x price increase',
        transform=ax.transAxes, ha='center', fontsize=13, color=RED,
        bbox=dict(boxstyle='round,pad=0.5', facecolor=CARD, edgecolor=RED, alpha=0.9))
plt.tight_layout()
plt.show()

**Interpretation:** A guest paying EUR 5,000/night gets a rating only 0.28 points higher than someone paying EUR 500. The Spearman correlation (0.19) confirms the relationship is weak. Price is driven primarily by location and brand prestige, not by actual guest satisfaction.

---
## 5. Finding 2 — Rooms Are the Strongest Point for 88% of Hotels

Booking.com provides both an overall rating and a separate room score. What's the gap?

In [ ]:
has_both = booking.dropna(subset=['room_score']).copy()
has_both['gap'] = has_both['room_score'] - has_both['rating']

room_higher = (has_both['gap'] > 0).sum()
room_equal = (has_both['gap'] == 0).sum()
room_lower = (has_both['gap'] < 0).sum()

print(f'Hotels analyzed: {len(has_both)}')
print(f'Room > Overall: {room_higher} ({room_higher/len(has_both)*100:.0f}%)')
print(f'Room = Overall: {room_equal} ({room_equal/len(has_both)*100:.0f}%)')
print(f'Room < Overall: {room_lower} ({room_lower/len(has_both)*100:.0f}%)')
print(f'Mean gap: +{has_both["gap"].mean():.2f}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), gridspec_kw={'width_ratios': [1, 1.4]})

sizes = [room_higher, room_equal, room_lower]
pie_colors = [ACCENT, YELLOW, RED]
wedges, texts, _ = ax1.pie(sizes, colors=pie_colors, startangle=90, autopct='',
                            textprops={'color': '#FFFFFF', 'fontsize': 11},
                            pctdistance=0.75, labeldistance=1.15)
labels_pie = [
    f'Room > Overall\n{room_higher} ({room_higher/len(has_both)*100:.0f}%)',
    f'Equal\n{room_equal} ({room_equal/len(has_both)*100:.0f}%)',
    f'Room < Overall\n{room_lower} ({room_lower/len(has_both)*100:.0f}%)']
for text, label in zip(texts, labels_pie):
    text.set_text(label); text.set_fontsize(11)
ax1.set_title('Room Score vs Overall Rating', fontsize=16, fontweight='bold', pad=15)

ax2.hist(has_both['gap'], bins=40, color=ACCENT, alpha=0.8, edgecolor='none')
ax2.axvline(0, color=RED, linestyle='--', linewidth=2, label='No gap (0)')
ax2.axvline(has_both['gap'].mean(), color=YELLOW, linestyle='--', linewidth=2,
            label=f'Mean gap: +{has_both["gap"].mean():.2f}')
ax2.set_title('88% of Hotels: Room Is the Strongest Point', fontsize=16, fontweight='bold', pad=15)
ax2.set_xlabel('Gap (Room Score - Overall Rating)')
ax2.set_ylabel('Number of Hotels')
ax2.legend(facecolor=CARD, edgecolor='#1a2d4a', fontsize=11)
plt.tight_layout()
plt.show()

**Interpretation:** In 88% of cases, the room scores higher than the overall rating. The average gap is +0.30. This means what drags overall ratings down is typically service, cleanliness, location convenience, or value for money — not the room itself. For hotel operators, investing in staff training and non-room amenities would likely have the biggest impact.

---
## 6. Finding 3 — Paris Is the Worst Value; Thailand Dominates Best Value

I created a Value Index (rating / median price * 100) to identify destinations that deliver the best and worst value.

In [ ]:
loc_stats = (booking.groupby('location')
    .agg(median_price=('price_eur', 'median'), mean_rating=('rating', 'mean'),
         count=('hotel_name', 'count'))
    .reset_index())
loc_stats = loc_stats[loc_stats['count'] >= 10].copy()
loc_stats['value_index'] = (loc_stats['mean_rating'] / loc_stats['median_price']) * 100

best10 = loc_stats.nlargest(10, 'value_index')
worst10 = loc_stats.nsmallest(10, 'value_index')

fig, (ax_w, ax_b) = plt.subplots(1, 2, figsize=(16, 8))

worst_s = worst10.sort_values('value_index', ascending=True)
bars_w = ax_w.barh(worst_s['location'], worst_s['median_price'], color=RED, alpha=0.85, height=0.6)
for bar, rating in zip(bars_w, worst_s['mean_rating']):
    ax_w.text(bar.get_width() + 50, bar.get_y() + bar.get_height()/2,
              f'{rating:.1f}', va='center', fontsize=11, color=YELLOW, fontweight='bold')
ax_w.set_title('WORST Value', fontsize=16, fontweight='bold', pad=15, color=RED)
ax_w.set_xlabel('Median Price (EUR)')
ax_w.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))
ax_w.text(1.0, 1.02, 'Rating', transform=ax_w.transAxes, ha='right', fontsize=10, color=YELLOW, fontweight='bold')

best_s = best10.sort_values('value_index', ascending=True)
bars_b = ax_b.barh(best_s['location'], best_s['median_price'], color=ACCENT, alpha=0.85, height=0.6)
for bar, rating in zip(bars_b, best_s['mean_rating']):
    ax_b.text(bar.get_width() + 20, bar.get_y() + bar.get_height()/2,
              f'{rating:.1f}', va='center', fontsize=11, color=YELLOW, fontweight='bold')
ax_b.set_title('BEST Value', fontsize=16, fontweight='bold', pad=15, color=ACCENT)
ax_b.set_xlabel('Median Price (EUR)')
ax_b.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))
ax_b.text(1.0, 1.02, 'Rating', transform=ax_b.transAxes, ha='right', fontsize=10, color=YELLOW, fontweight='bold')

fig.suptitle('Paris vs Thailand: Where Does Your Money Go?', fontsize=22, fontweight='bold', color='#FFFFFF', y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

**Interpretation:** 6 of the 10 worst value destinations are Paris arrondissements — high prices, mediocre ratings. Thailand dominates the best value list with ratings of 8.2-8.6 at a fraction of the cost. This could partially reflect the dataset composition, but the pattern is consistent across multiple locations in each region.

---
## 7. Finding 4 — Overpriced Hotels Identified Through Residual Analysis

I fitted a linear model (rating vs price) and flagged hotels that fall far below the expected rating for their price.

In [ ]:
mask = booking['price_eur'] < 5000
coeffs = np.polyfit(booking.loc[mask, 'price_eur'], booking.loc[mask, 'rating'], 1)
bf = booking[mask].copy()
bf['expected'] = np.polyval(coeffs, bf['price_eur'])
bf['residual'] = bf['rating'] - bf['expected']

overpriced = bf[bf['num_reviews'] >= 50].nsmallest(7, 'residual')

fig, ax = plt.subplots(figsize=(14, 8))
ax.scatter(bf['price_eur'], bf['rating'], alpha=0.12, s=12, color=SUBTLE)
x_line = np.linspace(0, 5000, 100)
ax.plot(x_line, np.polyval(coeffs, x_line), color=YELLOW, linewidth=2, linestyle='--', label='Expected rating')
ax.scatter(overpriced['price_eur'], overpriced['rating'], color=RED, s=100, zorder=5,
           edgecolors='#FFFFFF', linewidth=0.8, label='Most overpriced')

offsets = [(250, 0.6), (-400, 0.8), (250, -0.8), (-400, -0.5), (300, 0.5), (-350, -0.7), (200, 0.4)]
for i, (_, row) in enumerate(overpriced.iterrows()):
    name = row['hotel_name'][:22]
    ox, oy = offsets[i % len(offsets)]
    ax.annotate(f"{name}\n{row['location']}\nRating: {row['rating']}",
                xy=(row['price_eur'], row['rating']),
                xytext=(row['price_eur'] + ox, row['rating'] + oy),
                fontsize=8, color=RED,
                arrowprops=dict(arrowstyle='->', color=RED, lw=0.8),
                bbox=dict(boxstyle='round,pad=0.3', facecolor=CARD, edgecolor=RED, alpha=0.85))

ax.set_title('Overpriced: High Price, Low Rating', fontsize=20, fontweight='bold', pad=20)
ax.set_xlabel('Price per Night (EUR)', fontsize=14)
ax.set_ylabel('Rating', fontsize=14)
ax.set_xlim(0, 5000); ax.set_ylim(1, 10.5)
ax.legend(facecolor=CARD, edgecolor='#1a2d4a', fontsize=11, loc='lower right')
plt.tight_layout()
plt.show()

**Interpretation:** Modern's Hotel in Paris charges EUR 2,523/night but has a 3.2 rating across 1,101 reviews — that's not a one-off bad experience, it's a pattern. The linear model (Rating = 0.00007 * Price + 8.06) shows that price has almost no predictive power, and the worst offenders are consistently in Paris and Singapore's budget districts.

---
## 8. Finding 5 — Budget Guests Write 2x Longer Reviews

This was the most surprising finding. Using TripAdvisor comment data, I measured the relationship between hotel price and review length.

In [ ]:
ta = tripadvisor.dropna(subset=['comment']).copy()
ta['comment_words'] = ta['comment'].str.split().str.len()

corr_c, p_c = stats.spearmanr(ta['price_eur'], ta['comment_words'])
print(f'Spearman correlation (price vs review length): r = {corr_c:.4f} (p = {p_c:.2e})')
print(f'This is a moderately strong negative correlation — highly significant.')

In [ ]:
ta['price_bracket'] = pd.cut(ta['price_eur'],
    bins=[0, 30, 60, 100, 200, 10000], labels=['<30', '30-60', '60-100', '100-200', '200+'])
cp = (ta.groupby('price_bracket', observed=True)
      .agg(mean_words=('comment_words', 'mean'), count=('hotel_name', 'count'))
      .reset_index())

fig, ax = plt.subplots(figsize=(12, 7))
colors_g = [ACCENT, '#2ECC71', YELLOW, '#E67E22', RED]
bars = ax.bar(cp['price_bracket'].astype(str), cp['mean_words'], color=colors_g, alpha=0.85, width=0.55)
for bar, words, count in zip(bars, cp['mean_words'], cp['count']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.8,
            f'{words:.0f} words', ha='center', fontsize=14, fontweight='bold', color='#FFFFFF')
    ax.text(bar.get_x() + bar.get_width()/2, 1.5,
            f'n={count}', ha='center', fontsize=10, color=SUBTLE)
ax.set_title('Cheaper Hotels Get Longer Reviews', fontsize=20, fontweight='bold', pad=20)
ax.set_xlabel('Price per Night (EUR)', fontsize=14)
ax.set_ylabel('Average Words per Review', fontsize=14)
ax.set_ylim(0, 35)
ax.text(0.5, 0.92, 'Budget guests write 2x more words than luxury guests  |  Spearman r = -0.43',
        transform=ax.transAxes, ha='center', fontsize=12, color=RED,
        bbox=dict(boxstyle='round,pad=0.5', facecolor=CARD, edgecolor=RED, alpha=0.9))
plt.tight_layout()
plt.show()

**Interpretation:** Guests at hotels under EUR 30/night write an average of 27 words per review, compared to just 12 words at hotels over EUR 200. The correlation is strong (r = -0.43) and highly significant (p < 0.001). Possible explanations:

1. **More variance at budget level** — cheaper hotels have more unpredictable quality, prompting detailed descriptions
2. **Expectations met at luxury level** — luxury guests get what they paid for, leading to shorter confirmatory reviews
3. **Community-driven behavior** — budget travelers may be more engaged in helping other budget-conscious travelers

For hotel operators, longer reviews from budget guests are a rich source of actionable feedback.

---
## 9. Conclusions

| Finding | Key Metric |
|---------|------------|
| Price is a poor predictor of quality | Spearman r = 0.19, only +0.28 rating difference across 10x price range |
| Rooms are the strongest asset | 88% of hotels score higher on rooms than overall |
| Paris is the worst value destination | 6 of 10 worst value locations are Parisian arrondissements |
| Overpriced hotels are identifiable | Residual analysis reveals consistent underperformers |
| Budget guests leave richer feedback | 2x longer reviews, Spearman r = -0.43 |

### Limitations
- Currency conversion (BDT to EUR) uses an approximate fixed rate
- The two datasets cover different market segments and cannot be directly compared on price
- The dataset may over-represent certain regions (Thailand, Paris)
- No temporal dimension — all data is a snapshot, not a trend over time

---

*Analysis by Giorgio Vernarecci — Data Analyst*  
*Dataset: [Kaggle - Hotel Dataset: Rates, Reviews & Amenities](https://www.kaggle.com/datasets/joyshil0599/hotel-dataset-rates-reviews-and-amenities5k)*  
*GitHub: [github.com/logiop](https://github.com/logiop)*